In [ ]:
# Api Key
from dotenv import load_dotenv
import openai
import os

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
original_create = openai.chat.completions.create

In [ ]:
import pprint

def pretty_print_file(file_name):
    with open(file_name, 'r') as f:
        pp = pprint.PrettyPrinter(indent=4, width = 150)
        pp.pprint(f.read())

In [ ]:
# Set Once and every call will create an AIConfig
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
openai.chat.completions.create = create_and_save_to_config()

# Basic Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to default path `aiconfig.json`
print("Chat Completion Response: ")
pprint.pprint(response)

print("\nGenerated AIConfig Json: ")
pretty_print_file('aiconfig.json')

In [ ]:
# Set Once and every call will create an AIConfig
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set file path
aiconfig_file_path = "my-second-aiconfig.json"
openai.chat.completions.create  = create_and_save_to_config(config_file_path=aiconfig_file_path)

# Basic Streaming Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": True,
            "messages": [
                {
                    "content": "Tell me a riveting story",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Creates a config saved to aiconfig_file_path = "my-second-aiconfig.json"
print("Chat Completion Streaming Response: ")
for iteration in response:
    chunk = iteration.get("choices",[{}])[0].get('delta',{}).get('content','')
    print(chunk, end = '')


print("\nGenerated AIConfig Json: ")
pretty_print_file('aiconfig.json')
from openai import *

In [ ]:
# Set Once and every call will create an AIConfig
from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set aiconfig
aiconfig = AIConfigRuntime.load("my-second-aiconfig.json") # load the second aiconfig we just created and use it
openai.chat.completions.create = create_and_save_to_config(aiconfig=aiconfig)

# Compounded Execution
completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config was previously loaded from 'my-second-aiconfig.json. New Prompt gets added to the AIConfig Object and also saved to the same file

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "messages": [
                {
                    "content": "Tell me a joke about apples",
                    "role": "user",
                },
                 {
        "role": "assistant",
        "content": "Why did the apple go to school?\n\nBecause it wanted to be a \"smarty-pie\"!"
      },                                
      {
                    "content": "Tell this joke in a shakespearean rhetoric",
                    "role": "user",
                }
            ],
        }

response = openai.chat.completions.create(**completion_params) # Config is updated with the second prompt keeping the history intact



print("\nGenerated AIConfig Json: ")
pretty_print_file('my-second-aiconfig.json') # open the config yourself and check it out!

In [ ]:
# Function Call Capture
# Set Once and every call will create an AIConfig
from aiconfig.Config import AIConfigRuntime
import openai
from aiconfig.ChatCompletion import create_and_save_to_config
# Set aiconfig
openai.chat.completions.create = create_and_save_to_config(config_file_path='function_call.json')

def get_current_weather(location, unit):
    return { "temperature": 22, "unit": "celsius", "description": "Sunny" }

completion_params = {
    "model": "gpt-3.5-turbo-0613",
    "messages": [{"role": "user", "content": "What is the weather like in Boston?"}],
    "functions": [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ],
}

response = openai.chat.completions.create(**completion_params) 

function_call_response = get_current_weather(location="Boston", unit="celsius")
print(response)

completion_params = {
  "model": "gpt-3.5-turbo-0613",
  "messages": [
    {"role": "user", "content": "What is the weather like in Boston?"},
    {"role": "assistant", "content": 'null', "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }},
    {"role": "function", "name": "get_current_weather", "content": str(function_call_response)}

  ],
  "functions": [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
  ]
}

response = openai.chat.completions.create(**completion_params) 
print(response)

In [ ]:
import openai

# Instantiate a client
from aiconfig.ChatCompletion import create_and_save_to_config

client = openai.Client()
client.chat.completions.create = create_and_save_to_config()

completion_params = {
            "model": "gpt-3.5-turbo",
            "top_p": 1,
            "max_tokens": 3000,
            "temperature": 1,
            "stream": False,
            "messages": [
                {
                    "content": "Hi 2",
                    "role": "user",
                }
            ],
        }
response = client.chat.completions.create(**completion_params)


In [ ]:
from aiconfig import AIConfigRuntime
from aiconfig import InferenceOptions
# Reset Chat Completion to Original
from aiconfig.ChatCompletion import openai_chat_completion_create
openai.chat.completions.create = openai_chat_completion_create


config = AIConfigRuntime.load("aiconfig.json")
inference_options = InferenceOptions()
await config.run("prompt_0", options = inference_options)